<a href="https://colab.research.google.com/github/amitsangani/llama_index/blob/main/llama_doc_store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install llama-index

In [27]:
import os
os.environ['OPENAI_API_KEY'] = "YOUR_OPEN_API_KEY"

In [28]:
import nest_asyncio
nest_asyncio.apply()


In [29]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [30]:
# from gpt_index.composability.joint_qa_summary import QASummaryGraphBuilder
from llama_index import SimpleDirectoryReader, ServiceContext, LLMPredictor
from llama_index import GPTSimpleVectorIndex, GPTListIndex, GPTSimpleKeywordTableIndex
from llama_index.composability import ComposableGraph
from langchain.chat_models import ChatOpenAI


In [31]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [49]:
# read from data directory into documents object
reader = SimpleDirectoryReader('./data')
documents = reader.load_data()


In [50]:
# transfer documents to nodes
from llama_index.node_parser import SimpleNodeParser
nodes = SimpleNodeParser().get_nodes_from_documents(documents)


In [51]:
# add nodes to document store

from llama_index.docstore import DocumentStore
docstore = DocumentStore()
docstore.add_documents(nodes)

In [52]:
# define multiple indexes
# each index uses the same underlying node

list_index = GPTListIndex(nodes, docstore=docstore)

In [53]:
vector_index = GPTSimpleVectorIndex(nodes, docstore=docstore) 


In [54]:
list_index

In [55]:
list_index.index_struct

IndexList(index_id='26358f50-b936-477c-8414-95bcd88e8502', summary=None, nodes=['99a27c04-6dee-45ab-9a46-bf0407957b08', '4a702ef2-e0a9-4d94-b02e-730338131400', '322ec38c-69ea-4519-86f5-cfe1f2e92c7c'])

In [56]:
keyword_table_index = GPTSimpleKeywordTableIndex(nodes, docstore=docstore) 


In [57]:
# NOTE: the docstore still has the same nodes
len(docstore.docs)


3

In [58]:
llm_predictor_chatgpt = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-4"))
service_context_chatgpt = ServiceContext.from_defaults(llm_predictor=llm_predictor_chatgpt, chunk_size_limit=1024)


In [59]:
response = list_index.query("What is a summary of this document?") 


In [60]:
print(response)



Aladdin is a poor tailor's son who is idle and does not take care of his mother. One day, a stranger claiming to be his uncle arrives and offers to buy him a shop. The uncle then takes Aladdin to a mountain, where he reveals a hidden treasure and tells Aladdin to bring him a lamp from it. Aladdin obtains the lamp and discovers it is magical, with a genie who grants him wishes. Aladdin's mother sells the lamp to buy food, and Aladdin uses the genie to bring him a feast. When the Sultan's daughter is to be married, Aladdin uses the genie to transport her and her bridegroom to his home. The Sultan is impressed by Aladdin's wealth and agrees to the marriage, but sets a condition that Aladdin must bring him forty basins of gold filled with jewels. Aladdin uses the genie to fulfill the condition, summoning the genie to provide eighty slaves, richly dressed with jewels, to carry the basins of gold to the palace. Aladdin also uses the genie to provide himself and his mother with luxurious cl

In [61]:
response = vector_index.query("Who gave Aladin a magic lamp?") 
print(response)



The magic lamp was given to Aladdin by an African magician who claimed to be his uncle and summoned the genie to provide Aladdin with a scented bath, a richly embroidered habit, a horse surpassing the Sultan's, and twenty slaves to attend him, as well as six slaves to wait on his mother and ten thousand pieces of gold in ten purses.


In [62]:
response = keyword_table_index.query("What was the relationship between Sultan and Aladdin?")
print(response)



The Sultan and Aladdin had no prior relationship. The Sultan initially did not know Aladdin, but eventually agreed to give his daughter's hand in marriage to Aladdin after Aladdin's mother presented the Sultan with forty basins of gold filled with jewels and eighty slaves, all of whom were richly dressed and carrying splendid jewels. Aladdin also asked the genie for a scented bath, a richly embroidered habit, a horse surpassing the Sultan's, twenty slaves to attend him, six slaves to wait on his mother, and ten thousand pieces of gold in ten purses. The Sultan was so impressed by Aladdin's wealth and generosity that he agreed to the marriage. However, Aladdin's palace was stolen by an African magician, who wanted the Princess to marry him instead. Aladdin was able to rescue the Princess and the palace with the help of the genie of the lamp, and the Sultan was so pleased with Aladdin's courage and resourcefulness that he welcomed him back with open arms and allowed him to marry his da

In [45]:
response = keyword_table_index.query("Which school did the author go to?")
print(response)



The author went to Harvard for their undergraduate degree, Cornell for their graduate degree, the Rhode Island School of Design (RISD), and the Accademia di Belli Arti in Florence, where they started painting still lives in their bedroom at night.


In [63]:
response = keyword_table_index.query("Who was Fatima?")
print(response)


Fatima was a pious woman whom the African magician visited in China in order to avenge his brother's death. The magician disguised himself as Fatima and murdered her in order to prevent her from telling anyone about his plan. The magician then used Fatima's disguise to gain access to the palace of Aladdin, where he was mistaken for the holy woman.


In [65]:
response = keyword_table_index.query("Who was Aladdin's wife?")
print(response)



Aladdin's wife was the Sultan's daughter, the Princess. He was summoned to the palace to meet her and the Sultan, and he was given a scented bath, a richly embroidered habit, a horse surpassing the Sultan's, and twenty slaves to attend him, as well as six slaves to wait on his mother and ten thousand pieces of gold in ten purses. After being deceived by a wicked magician disguised as the holy woman Fatima, Aladdin and the Princess lived in peace and he eventually succeeded the Sultan when he died, and reigned for many years, leaving behind him a long line of kings. The Princess was charmed at the sight of Aladdin, who ran to receive her. "Princess," he said, "blame your beauty for my boldness if I have displeased you." She told him that, having seen him, she willingly obeyed her father in this matter. After the wedding had taken place, Aladdin led her into the hall, where a feast was spread, and she supped with him, after which they danced till midnight. Unfortunately, the wicked mag